# Healthcare Claims Fraud Detection - Exploratory Data Analysis (EDA)

## Overview
This notebook performs exploratory data analysis on healthcare claims data to understand patterns and prepare the dataset for fraud detection modeling. 

## Objectives
- Load and examine the raw healthcare claims dataset
- Handle missing values and data quality issues
- Remove unnecessary columns for fraud detection
- Clean and standardize categorical variables
- Handle outliers and anomalies
- Export cleaned data for feature engineering

## Dataset Information
- **Source**: Healthcare claims data
- **Target Variable**: `Is_Fraudulent` (Boolean indicating fraud)
- **Features**: Patient demographics, provider information, claim details, and behavioral patterns

## 1. Import Required Libraries
Import essential libraries for data manipulation and analysis.

In [280]:
# Core libraries for data manipulation and analysis
import pandas as pd
import numpy as np

# Display all columns in pandas
pd.set_option('display.max_columns', None)

## 2. Data Loading and Initial Exploration
Load the healthcare claims dataset and perform initial data inspection.

In [281]:
# Load the healthcare claims dataset
claims_df = pd.read_csv('../data/health_claims.csv')

# Display basic information about the dataset
print(f"Dataset shape: {claims_df.shape}")
print(f"Memory usage: {claims_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

Dataset shape: (20100, 30)
Memory usage: 20.90 MB


In [282]:
# Display first few rows to understand the data structure
claims_df.head()

,Patient_ID,Policy_Number,Claim_ID,Claim_Date,Service_Date,Policy_Expiration_Date,Claim_Amount,Patient_Age,Patient_Gender,Patient_City,Patient_State,Hospital_ID,Provider_Type,Provider_Specialty,Provider_City,Provider_State,Diagnosis_Code,Procedure_Code,Number_of_Procedures,Admission_Type,Discharge_Type,Length_of_Stay_Days,Service_Type,Deductible_Amount,CoPay_Amount,Number_of_Previous_Claims_Patient,Number_of_Previous_Claims_Provider,Provider_Patient_Distance_Miles,Claim_Submitted_Late,Is_Fraudulent
0,67349080,XAI956194550,18874,16-06-2021,01-05-2021,20-11-2024,462337.48,28,Female,Houston,WI,53,Laboratory,Cardiology,Los Angeles,WA,A09,93610,2,Elective,Rehab/Skilled Nursing,47,Inpatient,3440.06,824.59,0,16,939.08,True,False
1,85973291,XAI215993963,8762,19-01-2025,17-01-2025,07-04-2029,1883481.30,40,Other,Washington,CO,408,Laboratory,Cardiology,Dallas,AZ,J02.9,99203,2,Elective,Deceased,0,Outpatient,3618.40,851.43,0,6,170.40,False,True
2,62454860,XAI146653263,1920,05-05-2023,24-03-2023,04-09-2024,1500658.89,90,Female,Seattle,FL,106,Hospital,Neurology,Phoenix,NC,I25.10,71045,1,Urgent,Deceased,0,Pharmacy,2442.41,616.23,0,2,549.38,True,False
3,58632240,XAI216596381,1316,07-11-2021,02-11-2021,24-12-2025,572237.61,49,Other,Fort Worth,MD,933,Specialist Office,Pediatrics,Los Angeles,TX,I10,93571,4,Trauma,Deceased,0,Pharmacy,587.78,459.33,0,2,396.49,False,True
4,96347311,XAI056900359,17039,25-03-2024,18-02-2024,30-07-2028,2088293.84,24,Female,New York,FL,827,Specialist Office,Pediatrics,San Diego,PA,H25.9,93571,5,Emergency,Deceased,0,Emergency Room,3047.55,847.40,0,15,865.87,True,False


In [283]:
# Display all column names and data types
print("Dataset Columns:")
print(f"Total columns: {len(claims_df.columns)}")
print("\nColumn names:")
for i, col in enumerate(claims_df.columns, 1):
    print(f"{i:2d}. {col}")

print(f"\nData types:")
print(claims_df.dtypes)

Dataset Columns:
Total columns: 30

Column names:
 1. Patient_ID
 2. Policy_Number
 3. Claim_ID
 4. Claim_Date
 5. Service_Date
 6. Policy_Expiration_Date
 7. Claim_Amount
 8. Patient_Age
 9. Patient_Gender
10. Patient_City
11. Patient_State
12. Hospital_ID
13. Provider_Type
14. Provider_Specialty
15. Provider_City
16. Provider_State
17. Diagnosis_Code
18. Procedure_Code
19. Number_of_Procedures
20. Admission_Type
21. Discharge_Type
22. Length_of_Stay_Days
23. Service_Type
24. Deductible_Amount
25. CoPay_Amount
26. Number_of_Previous_Claims_Patient
27. Number_of_Previous_Claims_Provider
28. Provider_Patient_Distance_Miles
29. Claim_Submitted_Late
30. Is_Fraudulent

Data types:
Patient_ID                              int64
Policy_Number                          object
Claim_ID                                int64
Claim_Date                             object
Service_Date                           object
Policy_Expiration_Date                 object
Claim_Amount                          

## 3. Data Cleaning and Preprocessing

### 3.1 Remove Unnecessary Columns
Based on our analysis, we'll remove columns that don't contribute to fraud detection:
- **Patient_ID, Policy_Number, Claim_ID**: Pure identifiers with no predictive value
- **Patient_City, Provider_City**: Too granular, we'll keep state-level information
- **Policy_Expiration_Date**: Less relevant than claim timing patterns

In [284]:
# Define columns to remove (identifiers and overly granular geographic data)
columns_to_remove = [
    'Patient_ID',           # Patient identifier - no predictive value
    'Policy_Number',        # Policy identifier - no predictive value  
    'Claim_ID',            # Claim identifier - no predictive value
    'Policy_Expiration_Date',  # Less relevant than claim timing
    'Provider_City',       # Too granular - keeping state level
    'Patient_City'         # Too granular - keeping state level
]

print(f"Columns to remove ({len(columns_to_remove)}):")
for col in columns_to_remove:
    print(f"  - {col}")

Columns to remove (6):
  - Patient_ID
  - Policy_Number
  - Claim_ID
  - Policy_Expiration_Date
  - Provider_City
  - Patient_City


In [285]:
# claims_df.drop(columns_to_remove, axis=1,inplace=True)

In [286]:
claims_df.head()

,Patient_ID,Policy_Number,Claim_ID,Claim_Date,Service_Date,Policy_Expiration_Date,Claim_Amount,Patient_Age,Patient_Gender,Patient_City,Patient_State,Hospital_ID,Provider_Type,Provider_Specialty,Provider_City,Provider_State,Diagnosis_Code,Procedure_Code,Number_of_Procedures,Admission_Type,Discharge_Type,Length_of_Stay_Days,Service_Type,Deductible_Amount,CoPay_Amount,Number_of_Previous_Claims_Patient,Number_of_Previous_Claims_Provider,Provider_Patient_Distance_Miles,Claim_Submitted_Late,Is_Fraudulent
0,67349080,XAI956194550,18874,16-06-2021,01-05-2021,20-11-2024,462337.48,28,Female,Houston,WI,53,Laboratory,Cardiology,Los Angeles,WA,A09,93610,2,Elective,Rehab/Skilled Nursing,47,Inpatient,3440.06,824.59,0,16,939.08,True,False
1,85973291,XAI215993963,8762,19-01-2025,17-01-2025,07-04-2029,1883481.30,40,Other,Washington,CO,408,Laboratory,Cardiology,Dallas,AZ,J02.9,99203,2,Elective,Deceased,0,Outpatient,3618.40,851.43,0,6,170.40,False,True
2,62454860,XAI146653263,1920,05-05-2023,24-03-2023,04-09-2024,1500658.89,90,Female,Seattle,FL,106,Hospital,Neurology,Phoenix,NC,I25.10,71045,1,Urgent,Deceased,0,Pharmacy,2442.41,616.23,0,2,549.38,True,False
3,58632240,XAI216596381,1316,07-11-2021,02-11-2021,24-12-2025,572237.61,49,Other,Fort Worth,MD,933,Specialist Office,Pediatrics,Los Angeles,TX,I10,93571,4,Trauma,Deceased,0,Pharmacy,587.78,459.33,0,2,396.49,False,True
4,96347311,XAI056900359,17039,25-03-2024,18-02-2024,30-07-2028,2088293.84,24,Female,New York,FL,827,Specialist Office,Pediatrics,San Diego,PA,H25.9,93571,5,Emergency,Deceased,0,Emergency Room,3047.55,847.40,0,15,865.87,True,False


### 3.2 Missing Values Analysis
Before removing columns, let's check for missing values and handle them appropriately.

In [287]:
# Check for missing values in the dataset
null_counts = claims_df.isnull().sum()
null_percentages = (claims_df.isnull().sum() / len(claims_df)) * 100

print("Missing Values Analysis:")
print("-" * 40)
missing_data = pd.DataFrame({
    'Column': null_counts.index,
    'Missing_Count': null_counts.values,
    'Missing_Percentage': null_percentages.values
})
missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

if len(missing_data) > 0:
    print(missing_data.to_string(index=False))
else:
    print("No missing values found!")

Missing Values Analysis:
----------------------------------------
        Column  Missing_Count  Missing_Percentage
Provider_State             11            0.054726
 Patient_State              6            0.029851
  Patient_City              5            0.024876


In [288]:
# Identify rows with missing Patient_State values
null_patient_state = claims_df[claims_df['Patient_State'].isnull()]

print(f"Rows with missing Patient_State: {len(null_patient_state)}")
if len(null_patient_state) > 0:
    print("\nSample of rows with missing Patient_State:")
    print(null_patient_state[['Patient_City', 'Patient_State', 'Provider_City', 'Provider_State']].head())
    
    # Strategy: We can infer state from city when possible
    print("\nStrategy: Fill missing states using city-to-state mapping from existing data")

Rows with missing Patient_State: 6

Sample of rows with missing Patient_State:
     Patient_City Patient_State  Provider_City Provider_State
57        Phoenix           NaN        Seattle             IL
88   Indianapolis           NaN         Austin             IL
92         Denver           NaN  San Francisco             MN
226    Fort Worth           NaN       Columbus             MN
231       Chicago           NaN    San Antonio             GA

Strategy: Fill missing states using city-to-state mapping from existing data


In [289]:
# Example: Find the most common state for a specific city
# This demonstrates our approach to fill missing state values
example_city = 'New York'
if example_city in claims_df['Patient_City'].values:
    most_common_state = claims_df[claims_df['Patient_City'] == example_city]['Patient_State'].mode()
    if len(most_common_state) > 0:
        print(f"Most common state for {example_city}: {most_common_state[0]}")
    else:
        print(f"No state data available for {example_city}")
else:
    print(f"{example_city} not found in dataset")

Most common state for New York: TX


In [290]:
# Fill missing Patient_State values using city-to-state mapping
print("Filling missing Patient_State values...")
claims_df['Patient_State'] = claims_df.apply(
    lambda row: claims_df[claims_df['Patient_City'] == row['Patient_City']]['Patient_State'].mode()[0]
    if pd.isnull(row['Patient_State']) and row['Patient_City'] in claims_df['Patient_City'].unique()
    else row['Patient_State'],
    axis=1
)

# Fill missing Provider_State values using city-to-state mapping  
print("Filling missing Provider_State values...")
claims_df['Provider_State'] = claims_df.apply(
    lambda row: claims_df[claims_df['Provider_City'] == row['Provider_City']]['Provider_State'].mode()[0]
    if pd.isnull(row['Provider_State']) and row['Provider_City'] in claims_df['Provider_City'].unique()
    else row['Provider_State'],
    axis=1
)

print("Missing value imputation completed!")

Filling missing Patient_State values...
Filling missing Provider_State values...
Missing value imputation completed!
Filling missing Provider_State values...
Missing value imputation completed!


In [291]:
# Verify that missing values have been handled
print("Missing Values After Imputation:")
print("-" * 40)
remaining_nulls = claims_df.isnull().sum()
if remaining_nulls.sum() == 0:
    print("✅ All missing values have been successfully handled!")
else:
    print("⚠️ Some missing values remain:")
    print(remaining_nulls[remaining_nulls > 0])

Missing Values After Imputation:
----------------------------------------
⚠️ Some missing values remain:
Patient_City    5
dtype: int64


In [292]:
# Remove the identified unnecessary columns
print(f"Removing {len(columns_to_remove)} unnecessary columns...")
print(f"Original shape: {claims_df.shape}")

claims_df.drop(columns_to_remove, axis=1, inplace=True)

print(f"New shape: {claims_df.shape}")
print(f"Columns removed: {', '.join(columns_to_remove)}")

Removing 6 unnecessary columns...
Original shape: (20100, 30)
New shape: (20100, 24)
Columns removed: Patient_ID, Policy_Number, Claim_ID, Policy_Expiration_Date, Provider_City, Patient_City


### 3.3 Outlier Detection and Data Quality Assessment
Examine the data for outliers and quality issues that might affect our fraud detection model.

In [293]:
# Generate statistical summary for numerical features
print("Statistical Summary of Numerical Features:")
print("=" * 50)
numerical_summary = claims_df.describe()
print(numerical_summary)

print(f"\nDataset Info:")
print(f"Shape: {claims_df.shape}")
print(f"Numerical columns: {claims_df.select_dtypes(include=[np.number]).columns.tolist()}")
print(f"Categorical columns: {claims_df.select_dtypes(include=['object']).columns.tolist()}")

Statistical Summary of Numerical Features:
       Claim_Amount   Patient_Age   Hospital_ID  Procedure_Code  \
count  2.010000e+04  20100.000000  20100.000000    20100.000000   
mean   1.245349e+06     45.128010    500.204179    92463.849851   
std    6.983001e+05     26.341485    288.437500     8151.470654   
min    5.516780e+03      0.000000      1.000000    71045.000000   
25%    6.674103e+05     22.000000    251.000000    93040.000000   
50%    1.221707e+06     45.000000    500.000000    93571.000000   
75%    1.835603e+06     68.000000    752.000000    99204.000000   
max    2.499689e+06    200.000000   1000.000000    99284.000000   

       Number_of_Procedures  Length_of_Stay_Days  Deductible_Amount  \
count          20100.000000         20100.000000       20100.000000   
mean               5.510249             4.975821        2508.487758   
std                2.882847            13.239804        1443.579069   
min                1.000000             0.000000           0.030000  

In [294]:
# Display the cleaned dataset structure
print("Cleaned Dataset Overview:")
print(f"Shape: {claims_df.shape}")
print(f"Columns: {list(claims_df.columns)}")
print("\nFirst few rows of cleaned data:")
claims_df.head()

Cleaned Dataset Overview:
Shape: (20100, 24)
Columns: ['Claim_Date', 'Service_Date', 'Claim_Amount', 'Patient_Age', 'Patient_Gender', 'Patient_State', 'Hospital_ID', 'Provider_Type', 'Provider_Specialty', 'Provider_State', 'Diagnosis_Code', 'Procedure_Code', 'Number_of_Procedures', 'Admission_Type', 'Discharge_Type', 'Length_of_Stay_Days', 'Service_Type', 'Deductible_Amount', 'CoPay_Amount', 'Number_of_Previous_Claims_Patient', 'Number_of_Previous_Claims_Provider', 'Provider_Patient_Distance_Miles', 'Claim_Submitted_Late', 'Is_Fraudulent']

First few rows of cleaned data:


,Claim_Date,Service_Date,Claim_Amount,Patient_Age,Patient_Gender,Patient_State,Hospital_ID,Provider_Type,Provider_Specialty,Provider_State,Diagnosis_Code,Procedure_Code,Number_of_Procedures,Admission_Type,Discharge_Type,Length_of_Stay_Days,Service_Type,Deductible_Amount,CoPay_Amount,Number_of_Previous_Claims_Patient,Number_of_Previous_Claims_Provider,Provider_Patient_Distance_Miles,Claim_Submitted_Late,Is_Fraudulent
0,16-06-2021,01-05-2021,462337.48,28,Female,WI,53,Laboratory,Cardiology,WA,A09,93610,2,Elective,Rehab/Skilled Nursing,47,Inpatient,3440.06,824.59,0,16,939.08,True,False
1,19-01-2025,17-01-2025,1883481.30,40,Other,CO,408,Laboratory,Cardiology,AZ,J02.9,99203,2,Elective,Deceased,0,Outpatient,3618.40,851.43,0,6,170.40,False,True
2,05-05-2023,24-03-2023,1500658.89,90,Female,FL,106,Hospital,Neurology,NC,I25.10,71045,1,Urgent,Deceased,0,Pharmacy,2442.41,616.23,0,2,549.38,True,False
3,07-11-2021,02-11-2021,572237.61,49,Other,MD,933,Specialist Office,Pediatrics,TX,I10,93571,4,Trauma,Deceased,0,Pharmacy,587.78,459.33,0,2,396.49,False,True
4,25-03-2024,18-02-2024,2088293.84,24,Female,FL,827,Specialist Office,Pediatrics,PA,H25.9,93571,5,Emergency,Deceased,0,Emergency Room,3047.55,847.40,0,15,865.87,True,False


### 3.4 Categorical Variables Analysis and Standardization
Examine and clean categorical variables to ensure consistency.

In [295]:
# Analyze Patient_Gender distribution and identify inconsistencies
print("Patient_Gender Value Analysis:")
print("-" * 30)
gender_counts = claims_df.Patient_Gender.value_counts()
print(gender_counts)
print(f"\nUnique values: {claims_df.Patient_Gender.unique()}")
print(f"Total unique values: {claims_df.Patient_Gender.nunique()}")

Patient_Gender Value Analysis:
------------------------------
Patient_Gender
Female     6716
Other      6700
Male       6657
male         10
female        8
trans         8
Female        1
Name: count, dtype: int64

Unique values: ['Female' 'Other' 'Male' 'female' 'trans' 'male' 'Female ']
Total unique values: 7


In [296]:
# Inspect key categorical variables for inconsistencies
print("Categorical Variables Inspection:")
print("=" * 40)

print("\n1. Patient_Gender unique values:")
print(f"   {sorted(claims_df.Patient_Gender.unique())}")

print("\n2. Provider_Type unique values:")  
print(f"   {sorted(claims_df.Provider_Type.unique())}")

# Check for potential data quality issues (case sensitivity, extra spaces, etc.)
print(f"\n3. Potential data quality issues detected:")

Categorical Variables Inspection:

1. Patient_Gender unique values:
   ['Female', 'Female ', 'Male', 'Other', 'female', 'male', 'trans']

2. Provider_Type unique values:
   ['Clinic', 'Hospital', 'Laboratory', 'Pharmacy', 'Specialist Office', 'Urgent Care', 'clinic', 'laboratory']

3. Potential data quality issues detected:


In [297]:
# Standardize categorical values to ensure consistency
print("Standardizing categorical variables...")

# Standardize Patient_Gender values
print("\nBefore standardization - Patient_Gender:")
print(claims_df.Patient_Gender.value_counts())

claims_df.replace({
    'Patient_Gender': {
        'female': 'Female', 
        'Female ': 'Female',  # Remove extra space
        'male': 'Male', 
        'trans': 'Other'
    }
}, inplace=True)

# Standardize Provider_Type values  
print("\nBefore standardization - Provider_Type:")
print(claims_df.Provider_Type.value_counts())

claims_df.replace({
    'Provider_Type': {
        'laboratory': 'Laboratory',
        'clinic': 'Clinic'
    }
}, inplace=True)

print("\nStandardization completed!")

Standardizing categorical variables...

Before standardization - Patient_Gender:
Patient_Gender
Female     6716
Other      6700
Male       6657
male         10
female        8
trans         8
Female        1
Name: count, dtype: int64

Before standardization - Provider_Type:
Provider_Type
Specialist Office    5084
Pharmacy             3084
Urgent Care          3036
Clinic               2997
Hospital             2972
Laboratory           2910
laboratory             11
clinic                  6
Name: count, dtype: int64

Standardization completed!


In [298]:
# Verify standardization results
print("After standardization:")
print("\nPatient_Gender unique values:")
print(f"   {sorted(claims_df.Patient_Gender.unique())}")
print(f"   Value counts: {dict(claims_df.Patient_Gender.value_counts())}")

print("\nProvider_Type unique values:")
print(f"   {sorted(claims_df.Provider_Type.unique())}")  
print(f"   Value counts: {dict(claims_df.Provider_Type.value_counts())}")

After standardization:

Patient_Gender unique values:
   ['Female', 'Male', 'Other']
   Value counts: {'Female': 6725, 'Other': 6708, 'Male': 6667}

Provider_Type unique values:
   ['Clinic', 'Hospital', 'Laboratory', 'Pharmacy', 'Specialist Office', 'Urgent Care']
   Value counts: {'Specialist Office': 5084, 'Pharmacy': 3084, 'Urgent Care': 3036, 'Clinic': 3003, 'Hospital': 2972, 'Laboratory': 2921}


In [299]:
# Comprehensive analysis of all categorical variables
print("All Categorical Variables Summary:")
print("=" * 50)

categorical_columns = claims_df.select_dtypes(include=['object']).columns

for i, column in enumerate(categorical_columns, 1):
    unique_values = claims_df[column].unique()
    print(f"\n{i}. {column}:")
    print(f"   Unique count: {len(unique_values)}")
    if len(unique_values) <= 10:  # Show all values if 10 or fewer
        print(f"   Values: {sorted(unique_values)}")
    else:  # Show sample if more than 10
        print(f"   Sample values: {sorted(unique_values)[:10]}...")
        print(f"   Most frequent: {claims_df[column].value_counts().head(3).to_dict()}")

All Categorical Variables Summary:

1. Claim_Date:
   Unique count: 2019
   Sample values: ['01-01-2020', '01-01-2021', '01-01-2022', '01-01-2023', '01-01-2024', '01-01-2025', '01-02-2020', '01-02-2021', '01-02-2022', '01-02-2023']...
   Most frequent: {'29-06-2025': 22, '27-09-2020': 20, '23-06-2025': 20}

2. Service_Date:
   Unique count: 2093
   Sample values: ['01-01-2020', '01-01-2021', '01-01-2022', '01-01-2023', '01-01-2024', '01-01-2025', '01-02-2020', '01-02-2021', '01-02-2022', '01-02-2023']...
   Most frequent: {'10-10-2023': 22, '15-03-2023': 21, '06-01-2025': 20}

3. Patient_Gender:
   Unique count: 3
   Values: ['Female', 'Male', 'Other']

4. Patient_State:
   Unique count: 20
   Sample values: ['AZ', 'CA', 'CO', 'DC', 'FL', 'GA', 'IL', 'IN', 'MA', 'MD']...
   Most frequent: {'NY': 1054, 'TX': 1028, 'MN': 1027}

5. Provider_Type:
   Unique count: 6
   Values: ['Clinic', 'Hospital', 'Laboratory', 'Pharmacy', 'Specialist Office', 'Urgent Care']

6. Provider_Specialty:
   Un

### 3.5 Age Outlier Detection and Correction
Identify and handle unrealistic age values that could affect model performance.

In [300]:
# Analyze Patient_Age distribution for outliers
print("Patient Age Analysis:")
print("-" * 30)
age_stats = claims_df.Patient_Age.describe()
print(age_stats)

# Identify potential outliers (unrealistic ages)
print(f"\nAge Range Analysis:")
print(f"Minimum age: {claims_df.Patient_Age.min()}")
print(f"Maximum age: {claims_df.Patient_Age.max()}")
print(f"Ages > 100: {len(claims_df[claims_df.Patient_Age > 100])}")
print(f"Ages < 0: {len(claims_df[claims_df.Patient_Age < 0])}")

# Check for specific unrealistic values
unrealistic_ages = claims_df[claims_df.Patient_Age > 100]
if len(unrealistic_ages) > 0:
    print(f"\n⚠️ Found {len(unrealistic_ages)} records with unrealistic ages (>100)")
else:
    print("\n✅ No unrealistic ages found")

Patient Age Analysis:
------------------------------
count    20100.000000
mean        45.128010
std         26.341485
min          0.000000
25%         22.000000
50%         45.000000
75%         68.000000
max        200.000000
Name: Patient_Age, dtype: float64

Age Range Analysis:
Minimum age: 0
Maximum age: 200
Ages > 100: 4
Ages < 0: 0

⚠️ Found 4 records with unrealistic ages (>100)


In [301]:
# Examine records with unrealistic ages (>100)
unrealistic_age_records = claims_df[claims_df['Patient_Age'] > 100]

print(f"Records with Patient_Age > 100:")
print(f"Count: {len(unrealistic_age_records)}")

if len(unrealistic_age_records) > 0:
    print(f"\nSample of unrealistic age records:")
    display_cols = ['Patient_Age', 'Patient_Gender', 'Claim_Amount', 'Is_Fraudulent']
    print(unrealistic_age_records[display_cols].head(10))
    
    print(f"\nAge distribution for unrealistic values:")
    print(unrealistic_age_records['Patient_Age'].value_counts().sort_index())

Records with Patient_Age > 100:
Count: 4

Sample of unrealistic age records:
     Patient_Age Patient_Gender  Claim_Amount  Is_Fraudulent
85           200          Other    1803609.96          False
217          144          Other     701222.61          False
504          122          Other     493355.94          False
523          122          Other    2048581.74          False

Age distribution for unrealistic values:
Patient_Age
122    2
144    1
200    1
Name: count, dtype: int64


In [302]:
# Calculate mean age for realistic ages (< 100) to use for imputation
realistic_ages = claims_df[claims_df['Patient_Age'] < 100]['Patient_Age']
mean_age = realistic_ages.mean()

print(f"Age Imputation Strategy:")
print(f"Mean age for realistic values (< 100): {mean_age:.2f}")
print(f"Records to be corrected: {len(claims_df[claims_df['Patient_Age'] > 100])}")
print(f"Replacement value: {mean_age:.2f}")

Age Imputation Strategy:
Mean age for realistic values (< 100): 45.11
Records to be corrected: 4
Replacement value: 45.11


In [303]:
# Replace unrealistic ages with the calculated mean age
records_before = len(claims_df[claims_df['Patient_Age'] > 100])
print(f"Correcting {records_before} records with unrealistic ages...")

claims_df.loc[claims_df['Patient_Age'] > 100, 'Patient_Age'] = mean_age

records_after = len(claims_df[claims_df['Patient_Age'] > 100])
print(f"✅ Age correction completed!")
print(f"Records with age > 100 before: {records_before}")
print(f"Records with age > 100 after: {records_after}")
print(f"New age range: {claims_df.Patient_Age.min():.1f} - {claims_df.Patient_Age.max():.1f}")

Correcting 4 records with unrealistic ages...
✅ Age correction completed!
Records with age > 100 before: 4
Records with age > 100 after: 0
New age range: 0.0 - 90.0


C:\Users\Suraj Khodade\AppData\Local\Temp\ipykernel_12652\42670671.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '45.10773288216561' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  claims_df.loc[claims_df['Patient_Age'] > 100, 'Patient_Age'] = mean_age


In [304]:
# Verify that age correction was successful
verification_check = claims_df[claims_df['Patient_Age'] > 100]

print("Age Correction Verification:")
print("-" * 35)
if len(verification_check) == 0:
    print("✅ SUCCESS: No more records with age > 100")
    print(f"Current age statistics:")
    print(f"  Min: {claims_df.Patient_Age.min():.2f}")
    print(f"  Max: {claims_df.Patient_Age.max():.2f}")
    print(f"  Mean: {claims_df.Patient_Age.mean():.2f}")
    print(f"  Median: {claims_df.Patient_Age.median():.2f}")
else:
    print(f"⚠️ WARNING: Still have {len(verification_check)} records with age > 100")
    print(verification_check[['Patient_Age', 'Patient_Gender']].head())

Age Correction Verification:
-----------------------------------
✅ SUCCESS: No more records with age > 100
Current age statistics:
  Min: 0.00
  Max: 90.00
  Mean: 45.11
  Median: 45.00


## 4. Final Data Summary and Export

### 4.1 Cleaned Dataset Overview
Summary of the cleaned and processed dataset ready for feature engineering.

In [305]:
# Final cleaned dataset overview
print("📋 FINAL CLEANED DATASET SUMMARY")
print("=" * 50)

print(f"Dataset Shape: {claims_df.shape}")
print(f"Total Records: {len(claims_df):,}")
print(f"Total Features: {len(claims_df.columns)}")

print(f"\n📊 Target Variable Distribution:")
fraud_dist = claims_df['Is_Fraudulent'].value_counts()
fraud_pct = claims_df['Is_Fraudulent'].value_counts(normalize=True) * 100
for value, count in fraud_dist.items():
    print(f"  {'Fraudulent' if value else 'Non-Fraudulent'}: {count:,} ({fraud_pct[value]:.1f}%)")

print(f"\n🔢 Feature Types:")
numeric_cols = claims_df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = claims_df.select_dtypes(include=['object']).columns.tolist()
print(f"  Numerical features ({len(numeric_cols)}): {numeric_cols}")
print(f"  Categorical features ({len(categorical_cols)}): {categorical_cols}")

print(f"\n✅ Data Quality Checks:")
print(f"  Missing values: {claims_df.isnull().sum().sum()}")
print(f"  Duplicate rows: {claims_df.duplicated().sum()}")
print(f"  Age range: {claims_df.Patient_Age.min():.1f} - {claims_df.Patient_Age.max():.1f}")

print(f"\n📈 Ready for Feature Engineering!")

# Display sample of final data
print(f"\nSample of cleaned data:")
claims_df.head()

📋 FINAL CLEANED DATASET SUMMARY
Dataset Shape: (20100, 24)
Total Records: 20,100
Total Features: 24

📊 Target Variable Distribution:
  Non-Fraudulent: 15,089 (75.1%)
  Fraudulent: 5,011 (24.9%)

🔢 Feature Types:
  Numerical features (11): ['Claim_Amount', 'Patient_Age', 'Hospital_ID', 'Procedure_Code', 'Number_of_Procedures', 'Length_of_Stay_Days', 'Deductible_Amount', 'CoPay_Amount', 'Number_of_Previous_Claims_Patient', 'Number_of_Previous_Claims_Provider', 'Provider_Patient_Distance_Miles']
  Categorical features (11): ['Claim_Date', 'Service_Date', 'Patient_Gender', 'Patient_State', 'Provider_Type', 'Provider_Specialty', 'Provider_State', 'Diagnosis_Code', 'Admission_Type', 'Discharge_Type', 'Service_Type']

✅ Data Quality Checks:
  Missing values: 0
  Duplicate rows: 26  Duplicate rows: 26
  Age range: 0.0 - 90.0

📈 Ready for Feature Engineering!

Sample of cleaned data:

  Age range: 0.0 - 90.0

📈 Ready for Feature Engineering!

Sample of cleaned data:


,Claim_Date,Service_Date,Claim_Amount,Patient_Age,Patient_Gender,Patient_State,Hospital_ID,Provider_Type,Provider_Specialty,Provider_State,Diagnosis_Code,Procedure_Code,Number_of_Procedures,Admission_Type,Discharge_Type,Length_of_Stay_Days,Service_Type,Deductible_Amount,CoPay_Amount,Number_of_Previous_Claims_Patient,Number_of_Previous_Claims_Provider,Provider_Patient_Distance_Miles,Claim_Submitted_Late,Is_Fraudulent
0,16-06-2021,01-05-2021,462337.48,28.0,Female,WI,53,Laboratory,Cardiology,WA,A09,93610,2,Elective,Rehab/Skilled Nursing,47,Inpatient,3440.06,824.59,0,16,939.08,True,False
1,19-01-2025,17-01-2025,1883481.30,40.0,Other,CO,408,Laboratory,Cardiology,AZ,J02.9,99203,2,Elective,Deceased,0,Outpatient,3618.40,851.43,0,6,170.40,False,True
2,05-05-2023,24-03-2023,1500658.89,90.0,Female,FL,106,Hospital,Neurology,NC,I25.10,71045,1,Urgent,Deceased,0,Pharmacy,2442.41,616.23,0,2,549.38,True,False
3,07-11-2021,02-11-2021,572237.61,49.0,Other,MD,933,Specialist Office,Pediatrics,TX,I10,93571,4,Trauma,Deceased,0,Pharmacy,587.78,459.33,0,2,396.49,False,True
4,25-03-2024,18-02-2024,2088293.84,24.0,Female,FL,827,Specialist Office,Pediatrics,PA,H25.9,93571,5,Emergency,Deceased,0,Emergency Room,3047.55,847.40,0,15,865.87,True,False


In [306]:
# Display final dataset dimensions
print("Final Dataset Dimensions:")
print(f"Rows: {claims_df.shape[0]:,}")
print(f"Columns: {claims_df.shape[1]}")
print(f"Memory usage: {claims_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

Final Dataset Dimensions:
Rows: 20,100
Columns: 24
Memory usage: 15.46 MB

Rows: 20,100
Columns: 24
Memory usage: 15.46 MB


### 4.2 Export Cleaned Data
Save the cleaned dataset for the next phase: feature engineering.

In [307]:
# Export cleaned dataset for feature engineering
output_file = '../data/health_claims_eda.csv'

print(f"💾 Exporting cleaned dataset...")
print(f"Output file: {output_file}")
print(f"Records to export: {len(claims_df):,}")
print(f"Features to export: {len(claims_df.columns)}")

# Save the cleaned dataset
claims_df.to_csv(output_file, index=False)

# Verify export
import os
if os.path.exists(output_file):
    file_size = os.path.getsize(output_file) / 1024**2  # Size in MB
    print(f"✅ Export successful!")
    print(f"File size: {file_size:.2f} MB")
    print(f"Next step: Open notebook 02_feature_engineering.ipynb")
else:
    print("❌ Export failed!")

💾 Exporting cleaned dataset...
Output file: ../data/health_claims_eda.csv
Records to export: 20,100
Features to export: 24
✅ Export successful!
File size: 3.22 MB
Next step: Open notebook 02_feature_engineering.ipynb
✅ Export successful!
File size: 3.22 MB
Next step: Open notebook 02_feature_engineering.ipynb


## 5. EDA Summary and Insights

### Key Findings:
1. **Dataset Structure**: Successfully cleaned and processed healthcare claims dataset
2. **Missing Values**: Handled missing state values using city-to-state mapping
3. **Data Quality**: Standardized categorical variables and corrected unrealistic age values
4. **Target Variable**: Analyzed fraud distribution for class balance understanding
5. **Feature Preparation**: Removed unnecessary columns while preserving predictive features

### Data Cleaning Actions Completed:
- ✅ Removed 6 unnecessary columns (IDs and granular geographic data)
- ✅ Filled missing state values using existing city-state relationships
- ✅ Standardized categorical variables (gender, provider type)
- ✅ Corrected unrealistic age outliers (>100) with mean imputation
- ✅ Verified data quality and consistency

### Next Steps:
1. **Feature Engineering**: Create new features from existing data
2. **Exploratory Analysis**: Perform deeper statistical analysis and visualization
3. **Model Preparation**: Prepare data for machine learning algorithms

The cleaned dataset is now ready for feature engineering phase!